**Audio Separation Loss**

$$\mathcal{L}_{AS} = \sum_f^{all frames}(\sum_n^{N+1}  A_{{sep}_{n,i}} - A_{{ori}_i})^2 $$

In [ ]:
### AS Loss

**Less Separation Loss**

$$ hann\_func(x) := 종 모양 func$$

spectral flatness * power 가 0 혹은 1에 가까울 때 손실률이 0에 가까워지게

In [ ]:
### LS Loss

**Audio Dominance Loss**

$$ \mathcal{L}_{AD} = \sum_{n=1}^N (-\log{spectral\_flatness(A_{sep_n})}) \cdot \frac{\langle [max\_pooling({AF}_n)] \cdot D_n \rangle}{\Vert [max\_pooling({AF}_n)] \Vert \cdot \Vert D_n \Vert}) $$

**Scoremap Loss**

$$ \mathcal{L}_{Sm} = -\log \frac{{MAX}(Score_{u,v})}{\sum^{N,H,W}_{m,u,v}Score_{m,u,v}} $$

**Contrastive Loss**

<img src = "./IMGs/Clip_Fig1.png">

Given a batch of N (image, text) pairs, CLIP is trained to predict which of the N × N possible (image, text) pairings across a batch actually occurred. To do this, CLIP learns a multi-modal embedding space by jointly training an image encoder and text encoder to maximize the cosine similarity of the image and text embeddings of the N real pairs in the batch while minimizing the cosine similarity of the embeddings of the N2 − N incorrect pairings. We optimize a symmetric cross entropy loss over these similarity scores. In Figure 3 we include pseudocode of the core of an implementation of CLIP. \\

<!-- <img src = "./IMGs/Clip_Fig2.png">{width="50%"} -->

$$ Pos : S_{i\rightarrow i} = avg(per\_pixel\_embedding_i \cdot mask\_embedding_i) \\ .
\\
Neg : S_{i\rightarrow j} = avg(per\_pixel\_embedding_i \cdot mask\_embedding_j) \\ . \\
\mathcal{L}_{C} = 
$$

6분 35초 참조